In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.stats as stats

In [ ]:
# import the data and view it
df = pd.read_csv('chart.csv')

df

## Mean and Std - incomplete data!

What to do about 

In [ ]:
recruitment_mean = df['Recruitment'].mean(skipna=True)
recruitment_std = df['Recruitment'].std(skipna=True)

survival_mean = df['Survival Rate'].mean(skipna=True)
survival_std = df['Survival Rate'].std(skipna=True)

# show the mean and standard deviation of the recruitment and survival rate
print("Recruitment Mean: ", recruitment_mean)
print("Recruitment Standard Deviation: ", recruitment_std)
print("Survival Rate Mean: ", survival_mean)
print("Survival Rate Standard Deviation: ", survival_std)

## Part 3 - Simulation using normal distributions

Randomly sample from normal distribution for both recruitment and survival rates for 25 years. 

In [ ]:
# parameters for the simulation

starting_population = 275
num_simulations = 10
num_years = 25

In [ ]:
# create a table to store the simulations

starting_populations = pd.Series(np.ones(num_simulations) * starting_population, name='Year 0')
simulations = pd.DataFrame(starting_populations)

# show the table
simulations

In [ ]:
# run the simulation

# create a random number generator
rng = np.random.default_rng(seed=3290751)

for i in range(1, num_years + 1):
    # calculate the recruitment
    recruitment = rng.normal(recruitment_mean, recruitment_std, size=num_simulations)
    
    # calculate the survival rate
    survival = rng.normal(survival_mean, survival_std, size=num_simulations)
    
    # calculate the new population
    new_population = simulations[f"Year {i - 1}"] * survival + recruitment
    
    # add the new population to the table
    simulations[f"Year {i}"] = new_population

# round everything to 1 decimal place
simulations = simulations.round(1)
# show the table
simulations

In [ ]:
# plot the trials (row by row)

years_index = [i for i in range(num_years + 1)]
for i in range(num_simulations):
    if i == 1: # only put in the label once
        plt.plot(years_index, simulations.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        plt.plot(years_index, simulations.iloc[i, :], color='tab:gray')

# plot the actual data
plt.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

plt.xlabel("Year")
plt.ylabel("Population")
plt.title("Population over Time (Using normal distribution)")
plt.legend()
plt.show()
plt.close()

## Bootstrapping!

There's a better way to run the simulations that we learned in Stats Inference?

Two approaches 
- Assume that recruitment and survival rate are _independent_, which means that we can randomly sample from each one separately as a proxy for the distribution of possible recruitment - survival rate pairs.
- Assume that recruitment and survival rate are _dependent_, which means that we must randomly sample pairs of data points as a proxy for the actual distribution of possibilities.

### Bootstrapping - independence assumption
Randomly sample from recruitment and survival rates separately.

In [ ]:
# create a table to store the simulations

starting_populations = pd.Series(np.ones(num_simulations) * starting_population, name='Year 0')
bootstrap_sims = pd.DataFrame(starting_populations)

# show the table
bootstrap_sims

In [ ]:
# run the simulation

# create a random number generator
rng = np.random.default_rng(seed=29365701)

# create version of data without NaNs
# TODO decide if we drop entire rows with NaN or just skip the specific NaNs
recruitment_for_bootstrap = df['Recruitment'].dropna()
survival_for_bootstrap = df['Survival Rate'].dropna()


for i in range(1, num_years + 1):
    # randomly sample from the recruitment
    recruitment = rng.choice(recruitment_for_bootstrap, size=num_simulations, replace=True)
    
    # randomly sample from the survival rate
    survival = rng.choice(survival_for_bootstrap, size=num_simulations, replace=True)
    
    # calculate the new population
    new_population = bootstrap_sims[f"Year {i - 1}"] * survival + recruitment
    
    # add the new population to the table
    bootstrap_sims[f"Year {i}"] = new_population

# round everything to 1 decimal place
bootstrap_sims = bootstrap_sims.round(1)
# show the table
bootstrap_sims

In [ ]:
# plot the trials (row by row)

years_index = [i for i in range(num_years + 1)]
for i in range(num_simulations):
    if i == 1: # only put in the label once
        plt.plot(years_index, bootstrap_sims.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        plt.plot(years_index, bootstrap_sims.iloc[i, :], color='tab:gray')

# plot the actual data
plt.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

plt.xlabel("Year")
plt.ylabel("Population")
plt.title("Population over Time (Bootstrap method, assume independence)")
plt.legend()
plt.show()
plt.close()

### Bootstrapping - dependence assumption
Randomly sample from the pairs of data.

In [ ]:
# create a table to store the simulations

starting_populations = pd.Series(np.ones(num_simulations) * starting_population, name='Year 0')
bootstrap_sims_dep = pd.DataFrame(starting_populations)

# show the table
bootstrap_sims_dep

In [ ]:
# run the simulation

# create a random number generator
rng = np.random.default_rng(seed=29365701)

# create version of data without NaNs
df_for_bootstrap = df.dropna()
num_pairs = len(df_for_bootstrap.index)


for i in range(1, num_years + 1):
    # randomly sample from the recruitment and survival rate simulataneously
    pairs = rng.choice(num_pairs, size=num_simulations, replace=True)

    # get the recruitment and survival rate from each pair
    recruitment = [df_for_bootstrap['Recruitment'].iloc[pair] for pair in pairs]
    survival = [df_for_bootstrap['Survival Rate'].iloc[pair] for pair in pairs]

    # Convert to pandas series for vectorized operations
    recruitment = pd.Series(recruitment)
    survival = pd.Series(survival)
    
    # calculate the new population
    new_population = bootstrap_sims_dep[f"Year {i - 1}"] * survival + recruitment
    
    # add the new population to the table
    bootstrap_sims_dep[f"Year {i}"] = new_population

# round everything to 1 decimal place
bootstrap_sims_dep = bootstrap_sims_dep.round(1)
# show the table
bootstrap_sims_dep

In [ ]:
# plot the trials (row by row)

years_index = [i for i in range(num_years + 1)]
for i in range(num_simulations):
    if i == 1: # only put in the label once
        plt.plot(years_index, bootstrap_sims_dep.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        plt.plot(years_index, bootstrap_sims_dep.iloc[i, :], color='tab:gray')

# plot the actual data
plt.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

plt.xlabel("Year")
plt.ylabel("Population")
plt.title("Population over Time (Bootstrap method, assume dependence)")
plt.legend()
plt.show()
plt.close()

### Compare simulations

Just put all the graphs next to each other for easy comparison

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

years_index = [i for i in range(num_years + 1)]

# plot the normal distribution simulation

for i in range(num_simulations):
    if i == 1: # only put in the label once
        ax1.plot(years_index, simulations.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        ax1.plot(years_index, simulations.iloc[i, :], color='tab:gray')

# plot the actual data
ax1.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

ax1.set_xlabel("Year")
ax1.set_ylabel("Population")
ax1.set_title("Using normal distribution")
ax1.legend()

# plot the bootstrap simulation (independence)

for i in range(num_simulations):
    if i == 1: # only put in the label once
        ax2.plot(years_index, bootstrap_sims.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        ax2.plot(years_index, bootstrap_sims.iloc[i, :], color='tab:gray')

# plot the actual data
ax2.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

ax2.set_xlabel("Year")
ax2.set_title("Bootstrap method, assume independence")
ax2.legend()

# plot the bootstrap simulation (dependence)

for i in range(num_simulations):
    if i == 1: # only put in the label once
        ax3.plot(years_index, bootstrap_sims_dep.iloc[i, :], color='tab:gray', label='Simulations')
    else:
        ax3.plot(years_index, bootstrap_sims_dep.iloc[i, :], color='tab:gray')

# plot the actual data
ax3.plot(df['Actual Population'], color='tab:orange', linewidth=3, label='Actual Population')

ax3.set_xlabel("Year")
ax3.set_title("Bootstrap method, assume dependence")
ax3.legend()

plt.suptitle("Population over Time")
plt.show()
plt.close()



## Part 4 - Is it Normal?

Running a quantile comparison test to determine normalcy.  

In [ ]:
#quantile comparison on population input (recruitment, likely driven by birth rate)

#should show line of best fit if the data is shorn at the edges

stats.probplot(df['Recruitment'].dropna(), dist = "norm", plot=pylab)
plt.title('Recruitment Normalcy Test')
plt.show()

plt.hist(df['Recruitment'].dropna(), density=True, bins=30)
x = np.linspace(-50, 250, 1000)
y = stats.norm.pdf(x, loc=recruitment_mean, scale=recruitment_std)
plt.plot(x, y)

pylab.show()

#quantile comparison on death rate (given as percent survival)

stats.probplot(df['Survival Rate'].dropna(), dist = "norm", plot=pylab)
plt.title('Survival Rate Normalcy Test')
pylab.show()

plt.hist(df['Survival Rate'].dropna(), density=True, bins=20)
x = np.linspace(0.3, 1, 1000)
y = stats.norm.pdf(x, loc=survival_mean, scale=survival_std)
plt.plot(x, y)

plt.show()